In [1]:
# here, we convert monthly-emissions-1996-2024.csv into the proper timer series format 
# and output monthly-emissions-1996-2024-final.csv
# we also incorporate 1990-1995 data from monthly_emissions_1990_1996.csv

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df96 = pd.read_csv('../../SharedData/dataset-generation/monthly-emissions-1996-2024.csv')
df90 = pd.read_csv('../../SharedData/dataset-generation/monthly-emissions-1990-1996.csv')

In [2]:
df90['date']=df90['date'].apply(pd.to_datetime)
df90['date'] = df90['date'].dt.strftime('%Y-%m')
df90 = df90[['state','date','monthly_emissions']]

In [3]:
# reformat year and month within a single column of form yyyy-mm
df96['Month'] = df96['Month'].apply("{:02d}".format)
df96['date'] = df96['Year'].astype(str)+'-'+df96['Month'].astype(str)

# sum up CO2 emissions by state and date
df96 = df96.rename(columns={'State':'state'})
new_df = df96.groupby(['state', 'date'])['CO2 Mass (short tons)'].sum().reset_index(name='monthly_emissions')
new_df = new_df[~new_df['state'].isin(['AK','HI'])]

new_df['date'] = new_df['date'].apply(pd.to_datetime)
new_df['date'] = new_df['date'].dt.strftime('%Y-%m')
new_df = new_df[new_df['date'] >= '1997-01']

In [4]:
df_emissions = pd.concat([df90, new_df],axis=0)
df_emissions = df_emissions.sort_values(['state','date']).reset_index(drop=True)
df_emissions['date'] = df_emissions['date'].apply(pd.to_datetime)
df_emissions['date'] = df_emissions['date'].dt.strftime('%Y-%m')

In [ ]:
# abbreviations = [
#     "AL", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
#     "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
#     "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
#     "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
#     "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
# ]

# df_emissions['yearly_emissions'] = np.empty(df_emissions.shape[0])

# idx=0
# for state in abbreviations:
#     for year in range(1990,2024):
#         yr_emissions = df_emissions.loc[(df_emissions.state==state)&(df_emissions.date.dt.year==year)]['monthly_emissions'].sum()
#         yr_emissions_arr = np.empty(12)
#         yr_emissions_arr.fill(yr_emissions)
#         df_emissions[(df_emissions.state==state) & (df_emissions.date.dt.year==year)]['yearly_emissions'] = yr_emissions_arr


In [5]:
df_emissions.to_csv('../../SharedData/dataset-generation-final/monthly-emissions-1990-2024-final.csv', index=False)